In [1]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/esteban.berumen@iteso.mx/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-02.parquet')

In [35]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [36]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

In [37]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [38]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [39]:
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

In [40]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [41]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------
def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con early stopping en el conjunto de validación
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Predicción y métrica en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

In [42]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=3)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-23 21:15:08,750] A new study created in memory with name: no-name-523758b9-e290-4c57-8bb6-1d35bf6da5e8


[0]	validation-rmse:5.72427
[1]	validation-rmse:5.57860
[2]	validation-rmse:5.56409
[3]	validation-rmse:5.56982
[4]	validation-rmse:5.57347
[5]	validation-rmse:5.55585
[6]	validation-rmse:5.55736
[7]	validation-rmse:5.55253
[8]	validation-rmse:5.55232
[9]	validation-rmse:5.53322
[10]	validation-rmse:5.53156
[11]	validation-rmse:5.53006
[12]	validation-rmse:5.52808
[13]	validation-rmse:5.52782
[14]	validation-rmse:5.52451
[15]	validation-rmse:5.52324
[16]	validation-rmse:5.52269
[17]	validation-rmse:5.52229
[18]	validation-rmse:5.52366
[19]	validation-rmse:5.52900
[20]	validation-rmse:5.52988
[21]	validation-rmse:5.52969
[22]	validation-rmse:5.52872
[23]	validation-rmse:5.53195
[24]	validation-rmse:5.52894
[25]	validation-rmse:5.52702
[26]	validation-rmse:5.53066


c:\Users\esteb\apps\PC-20205\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:15:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/23 21:15:32 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\esteb\apps\PC-20205\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:15:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-23 21:15:34,808] Trial 0 finished with value: 5.5334711429183985 and parameters: {'max_depth': 40, 'learning_rate': 0.8625543817410922, 'reg_alpha': 0.12593061066249622, 'reg_lambda': 0.049454235173237264, 'min_child_weight': 0.6866535292359801}. Best is trial 0 with value: 5.5334711429183985.


🏃 View run salty-shrike-446 at: https://adb-3598518169245956.16.azuredatabricks.net/ml/experiments/3998396881915768/runs/34d00dd203ff4d638f059701d38dcb15
🧪 View experiment at: https://adb-3598518169245956.16.azuredatabricks.net/ml/experiments/3998396881915768
[0]	validation-rmse:8.77707
[1]	validation-rmse:8.47452
[2]	validation-rmse:8.19669
[3]	validation-rmse:7.94182
[4]	validation-rmse:7.70901
[5]	validation-rmse:7.49664
[6]	validation-rmse:7.30245
[7]	validation-rmse:7.12626
[8]	validation-rmse:6.96581
[9]	validation-rmse:6.81994
[10]	validation-rmse:6.68775
[11]	validation-rmse:6.56832
[12]	validation-rmse:6.46039
[13]	validation-rmse:6.36272
[14]	validation-rmse:6.27449
[15]	validation-rmse:6.19394
[16]	validation-rmse:6.12203
[17]	validation-rmse:6.05669
[18]	validation-rmse:5.99838
[19]	validation-rmse:5.94487
[20]	validation-rmse:5.89709
[21]	validation-rmse:5.85472
[22]	validation-rmse:5.81565
[23]	validation-rmse:5.78117
[24]	validation-rmse:5.74956
[25]	validation-rmse:5.72

...

c:\Users\esteb\apps\PC-20205\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:16:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-23 21:16:11,491] Trial 2 finished with value: 5.609300918544036 and parameters: {'max_depth': 5, 'learning_rate': 0.9136840519292247, 'reg_alpha': 0.18820387978911576, 'reg_lambda': 0.007166739666045858, 'min_child_weight': 0.7613210498541186}. Best is trial 1 with value: 5.410463874732254.


🏃 View run judicious-auk-873 at: https://adb-3598518169245956.16.azuredatabricks.net/ml/experiments/3998396881915768/runs/bf506fee6e814da889515d88803ae2bc
🧪 View experiment at: https://adb-3598518169245956.16.azuredatabricks.net/ml/experiments/3998396881915768
[0]	validation-rmse:8.77707
[1]	validation-rmse:8.47452
[2]	validation-rmse:8.19669
[3]	validation-rmse:7.94182
[4]	validation-rmse:7.70901
[5]	validation-rmse:7.49664
[6]	validation-rmse:7.30245
[7]	validation-rmse:7.12626
[8]	validation-rmse:6.96581
[9]	validation-rmse:6.81994
[10]	validation-rmse:6.68775
[11]	validation-rmse:6.56832
[12]	validation-rmse:6.46039
[13]	validation-rmse:6.36272
[14]	validation-rmse:6.27449
[15]	validation-rmse:6.19394
[16]	validation-rmse:6.12203
[17]	validation-rmse:6.05669
[18]	validation-rmse:5.99838
[19]	validation-rmse:5.94487
[20]	validation-rmse:5.89709
[21]	validation-rmse:5.85472
[22]	validation-rmse:5.81565
[23]	validation-rmse:5.78117
[24]	validation-rmse:5.74956
[25]	validation-rmse:5.7

c:\Users\esteb\apps\PC-20205\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:16:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/23 21:16:28 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\esteb\apps\PC-20205\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:16:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


🏃 View run XGBoost Hyperparameter Optimization (Optuna) at: https://adb-3598518169245956.16.azuredatabricks.net/ml/experiments/3998396881915768/runs/cdd16d4123a34a59be4e2ab1f2fa30b9
🧪 View experiment at: https://adb-3598518169245956.16.azuredatabricks.net/ml/experiments/3998396881915768


In [43]:
model_name = "time_series.default.nyc-taxi-model"

In [44]:
import mlflow
print("tracking:", mlflow.get_tracking_uri())
print("registry:", mlflow.get_registry_uri())

tracking: databricks
registry: databricks-uc


In [45]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="time_series.default.nyc-taxi-model"
)

Registered model 'time_series.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/23 21:16:38 WARNING mlflow.tracking._model_registry.fluent: Run with id cdd16d4123a34a59be4e2ab1f2fa30b9 has no artifacts at artifact path 'model', registering model based on models:/m-ceae3029e29a48b49cc4314656549cee instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '4' of model 'time_series.default.nyc-taxi-model'.


In [46]:
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    order_by=["metrics.rmse ASC"],
    output_format="list"
)

# Obtener el mejor run
if len(runs) > 0:
    best_run = runs[0]
    print("🏆 Champion Run encontrado:")
    print(f"Run ID: {best_run.info.run_id}")
    print(f"RMSE: {best_run.data.metrics['rmse']}")
    print(f"Params: {best_run.data.params}")
else:
    print("⚠️ No se encontraron runs con métrica RMSE.")

🏆 Champion Run encontrado:
Run ID: 391a4d75c8944b6aaf705c0df3fbfd62
RMSE: 5.410463874732254
Params: {'custom_metric': 'None', 'early_stopping_rounds': '10', 'learning_rate': '0.059264241587996896', 'max_depth': '19', 'maximize': 'None', 'min_child_weight': '6.248180561354165', 'num_boost_round': '100', 'objective': 'reg:squarederror', 'reg_alpha': '0.21539205131792016', 'reg_lambda': '0.05006540936006931', 'seed': '42', 'verbose_eval': 'True'}


In [47]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=model_name
)

Registered model 'time_series.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/23 21:16:48 WARNING mlflow.tracking._model_registry.fluent: Run with id 391a4d75c8944b6aaf705c0df3fbfd62 has no artifacts at artifact path 'model', registering model based on models:/m-b907456d6fed44a48dd47c64d16b2ffe instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '5' of model 'time_series.default.nyc-taxi-model'.


In [48]:
from mlflow import MlflowClient

client = MlflowClient()

In [49]:
model_version = result.version
new_alias = "Champion"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

In [50]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1761275813244, current_stage=None, deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 5 was transitioned to Champion on 2025-10-23 21:16:57.365518', last_updated_timestamp=1761275817791, metrics=[<Metric: dataset_digest='', dataset_name='', key='best_iteration', model_id='m-b907456d6fed44a48dd47c64d16b2ffe', run_id='391a4d75c8944b6aaf705c0df3fbfd62', step=0, timestamp=1761275743229, value=99.0>,
 <Metric: dataset_digest='', dataset_name='', key='rmse', model_id='m-b907456d6fed44a48dd47c64d16b2ffe', run_id='391a4d75c8944b6aaf705c0df3fbfd62', step=0, timestamp=1761275747069, value=5.410463874732254>,
 <Metric: dataset_digest='', dataset_name='', key='stopped_iteration', model_id='m-b907456d6fed44a48dd47c64d16b2ffe', run_id='391a4d75c8944b6aaf705c0df3fbfd62', s

In [51]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}@Champion"

champion_version = mlflow.pyfunc.load_model(model_version_uri)
champion_version.predict(X_val)

c:\Users\esteb\apps\PC-20205\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:17:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


array([ 5.182502, 25.113163, 28.840343, ..., 20.413898, 15.336824,
       22.998106], shape=(44218,), dtype=float32)